
# 基于修改后的 MindQuantum 0.5.0 采用变分算法重构半导体双量子点单-三重态量子 CZ 门

# 采用自定义的 MQLayer abs 来实现

In [5]:
import numpy as np
import sys
from numpy import kron
from mindquantum import *
from scipy.linalg import expm
import mindspore as ms
from mindspore import ops, Tensor
ms.context.set_context(mode=ms.context.PYNATIVE_MODE, device_target="CPU")
from mindspore.nn import Adam, TrainOneStepCell
from mindspore.common.parameter import Parameter
from mindspore.common.initializer import initializer  
ms.set_seed(1)
np.random.seed(1)

train_x = np.load('./src/cz_train_x.npy', allow_pickle=True)
eval_x = np.load('./src/cz_eval_x.npy', allow_pickle=True)
train_y = np.load('./src/cz_train_y.npy', allow_pickle=True)
eval_y = np.load('./src/cz_eval_y.npy', allow_pickle=True)

s_x = X.matrix()
s_z = Z.matrix()
one = I.matrix()
dt = np.pi/10

def _matrix_0(coeff):
    return expm(-1j*(coeff*s_z+s_x)*dt)

def _diff_matrix_0(coeff):
    return -1j*_matrix_0(coeff)@(s_z*dt)

def _matrix_c_0(coeff):
    return expm(-1j*(coeff*kron(s_z, one) + kron(one, s_z) + kron(s_x, one) + kron(one, s_x) + coeff*kron(s_z-one, s_z-one))*5*dt)

def _diff_matrix_c_0(coeff):
    return -1j*_matrix_c_0(coeff)@((kron(s_z, one) + kron(s_z-one, s_z-one)) * 5*dt)

def _matrix_c_1(coeff):
    return expm(-1j*(kron(s_z, one) + coeff*kron(one, s_z) + kron(s_x, one) + kron(one, s_x) + coeff*kron(s_z-one, s_z-one))*5*dt)

def _diff_matrix_c_1(coeff):
    return -1j*_matrix_c_1(coeff)@((kron(one, s_z) + kron(s_z-one, s_z-one)) *  5*dt)

gate_0 = gene_univ_parameterized_gate('gete_0', _matrix_0, _diff_matrix_0)
gate_c_0 = gene_univ_parameterized_gate('gete_c_0', _matrix_c_0, _diff_matrix_c_0)
gate_c_1 = gene_univ_parameterized_gate('gete_c_1', _matrix_c_1, _diff_matrix_c_1)

circ = Circuit()

circ += gate_0('00').on(0)
circ += gate_0('01').on(0)
circ += gate_0('02').on(0)
circ += gate_0('03').on(0)
circ += gate_0('04').on(0)
circ += gate_0('05').on(0)
circ += gate_0('06').on(0)
circ += gate_0('07').on(0)
circ += gate_0('08').on(0)
circ += gate_0('09').on(0)

circ += gate_0('10').on(1)
circ += gate_0('11').on(1)
circ += gate_0('12').on(1)
circ += gate_0('13').on(1)
circ += gate_0('14').on(1)
circ += gate_0('15').on(1)
circ += gate_0('16').on(1)
circ += gate_0('17').on(1)
circ += gate_0('18').on(1)
circ += gate_0('19').on(1)

circ += gate_c_0('0').on([1,0])
circ += gate_c_0('1').on([1,0])
circ += gate_c_1('2').on([1,0])
circ += gate_c_1('3').on([1,0])

circ += gate_0('010').on(0)
circ += gate_0('011').on(0)
circ += gate_0('012').on(0)
circ += gate_0('013').on(0)
circ += gate_0('014').on(0)
circ += gate_0('015').on(0)
circ += gate_0('016').on(0)
circ += gate_0('017').on(0)
circ += gate_0('018').on(0)
circ += gate_0('019').on(0)

circ += gate_0('110').on(1)
circ += gate_0('111').on(1)
circ += gate_0('112').on(1)
circ += gate_0('113').on(1)
circ += gate_0('114').on(1)
circ += gate_0('115').on(1)
circ += gate_0('116').on(1)
circ += gate_0('117').on(1)
circ += gate_0('118').on(1)
circ += gate_0('119').on(1)

ham = Hamiltonian(QubitOperator('')) 
sim = Simulator('mqvector', circ.n_qubits)
sim_left = Simulator('mqvector',circ.n_qubits)
grad_ops = sim.get_expectation_with_grad(ham,
                                         circ,
                                         circ_left=Circuit(),
                                         simulator_left=sim_left,
                                         ansatz_params_name=circ.params_name)
lr = 0.02
Quantum_net = MQLayer(grad_ops)
opti = Adam(Quantum_net.trainable_params(), learning_rate=lr)  
net = TrainOneStepCell(Quantum_net, opti)
error_min = 1

for j in range(len(train_x)):
    net(Tensor(train_x[j]), Tensor(train_y[j]))
    params = abs(Quantum_net.weight.asnumpy())
    final_state = []
    for k in range(100): # 100 个测试点
        sim.reset()
        sim.set_qs(eval_x[k])
        sim.apply_circuit(circ, params)
        final_state.append(sim.get_qs())
    error = 1-np.real(np.min([np.abs(np.vdot(bra, ket)) for bra, ket in zip(np.array(final_state), eval_y)]))
    if error < error_min:
        error_min = error
        params_tem = params
        j_tem = j
    if j % 50 == 0:
        print('error_min:', error_min, 'error_now:', error, 'train_num:', j)
    if error_min < 1e-5:
        break
        
print('\nerror_min:', error_min, 'train_num:', j_tem)
print('params_tem:\n', params_tem)

error_min: 0.7284220873912217 error_now: 0.7284220873912217 train_num: 0
error_min: 0.4659930631862742 error_now: 0.5032378009903851 train_num: 50
error_min: 0.32920229189059214 error_now: 0.41045665838040957 train_num: 100
error_min: 0.14859649090207938 error_now: 0.14859649090207938 train_num: 150
error_min: 0.12639008657954098 error_now: 0.19375665229527717 train_num: 200
error_min: 0.050129373919741815 error_now: 0.050129373919741815 train_num: 250
error_min: 0.041020447415039296 error_now: 0.047751516466245536 train_num: 300
error_min: 0.03726347991238632 error_now: 0.04683054248060836 train_num: 350
error_min: 0.02930082262765077 error_now: 0.030186070786646146 train_num: 400
error_min: 0.022283901801077755 error_now: 0.02265701744097559 train_num: 450
error_min: 0.02117732176026199 error_now: 0.022497318745653194 train_num: 500
error_min: 0.018884560299317288 error_now: 0.024477606917692962 train_num: 550
error_min: 0.018884560299317288 error_now: 0.028055087762377195 train_num: